<center><img src="../Picture Data/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

# Real-time display camera
Initialize the camera and display the screen you see.

Our neural network uses 224 x 224 pixel images as input. So we set the camera to this size to minimize the file size and minimize the data set. (We have tested this pixel for this task.) In some cases, it is best to use a larger image size when collecting data, and then reduce it to the required size when processing.

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)

image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

We need to create some directories to store the data. We will create a folder called dataset with two sub-folders, which is free and blocked, for sorting the images of each scene.

In [ ]:
import os

blocked_dir = 'dataset/blocked'
free_dir = 'dataset/free'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

If you refresh the Jupyter file browser on the left, you should now see these directories appear. Next, let's create and display some buttons that will be used to save a snapshot for each class tag. We'll also add some text boxes that will display the number of images for each category we've collected so far. This is useful because we want to make sure that we collect as many "free" images as "block" images. It also helps to understand how many images we have collected in total.

In [ ]:
button_layout = widgets.Layout(width='128px', height='64px')
free_button = widgets.Button(description='add free', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='add blocked', button_style='danger', layout=button_layout)
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))

display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

Now, these buttons can't do anything. We have to attach a function to save the image for each category of buttons ''n_click''. We will save the value of the ''Image'' widget (not the camera) because it is already in compressed JPEG format!

To make sure that no filenames are repeated (even on different machines!), we'll use the ''uuid'' package in python, which defines the ''uuid1'' method to generate a unique identifier. This unique identifier is generated from information such as current time and machine address.

In [ ]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))
    
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())

Now, the button above can already save the image to the "free" and "blocked" directories. You can view these files using the Jupyter left directory file browser!

Now continue to collect some data:
1. Place the robot in a scene where it is blocked and press "add blocked".
2. Place the robot in a free scene and press "add free".
3. Repeat 1, 2
(Reminder: You can move the widget to a new window by right-clicking the cell and clicking Create new View for Output.)

Here are some tips for tagging data:
Try different directions
2. Try different lighting
3. Try different object/conflict types; walls, rock shelves, objects
4. Try floors/objects of different textures; patterned, smooth, glass, etc.

It's important to get a variety of data (as described in the tips above), not just a lot of data, and you might need at least 100 images per class (this is not science, just a useful trick).

After running the cell code below, the image and button will be displayed and you can start collecting data:

In [ ]:
display(image)
display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

When you collect enough data, we need to copy this data to our GPU platform for training.

First, we can call the terminal (command line mode or terminal) command to compress the data into a *.zip file.

! means that we want to use the shell command to run； -r to include all subfolder files； -q means the zip command does not output any information.

In [ ]:
!zip -r -q dataset.zip dataset

When you run the code in the above cell, you will see a file named dataset.zip in the Jupyter Lab file browser. You can right click on the file to download it.